<a href="https://colab.research.google.com/github/shivendrra/SmallLanguageModel-project/blob/main/Karpathy_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Code is kind of replica of Karpathy's GPT that he made tutorial about

In [1]:
with open('training_data.txt', 'r', encoding='utf-8') as file:
  text = file.read()

In [ ]:
# all unique characters that are present in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [ ]:
#  encoder and decoder of the text
string_to_index = { ch:i for i,ch in enumerate(chars) }
index_to_string = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [string_to_index[c] for c in s]
decode = lambda l: ''.join([index_to_string[i] for i in l])

print(encode('hello there'))
print(encode('hello There'))
print(decode(encode('hello there')))

In [ ]:
# tokenizing the entire dataset
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

In [5]:
# train-test spliting
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the output is {target}")

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_block(split):
  data = train_data if split == 'train' else test_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_block('train')
print('inputs')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context} the output is {target}")


In [22]:
# bigram model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, index, targets):
    logits = self.token_embedding_table(index) # B, T, C but pyTorch expects input as B, C, T

    if targets is None:
      loss = None

    else:
      B, T, C = logits.shape
      logits = logits.view(B * T, C)
      targets = targets.view(B * T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(index, None) # get the predictions
      logits = logits[: ,-1, :] # focus on the last step // becomes (B, C)

      probs = F.softmax(logits, dim=-1) # applying softmax to get the probabilities // (B, C)
      index_next = torch.multinomial(probs, num_samples=1) # sample from the distribution // (B, 1)

      index = torch.cat((index, index_next), dim=1) # append sampled indexes to the running sequence // (B, T+1)

    return index


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss) # loss should be around 5.9 for this training data

print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 411])
tensor(6.6335, grad_fn=<NllLossBackward0>)

ｉ͂℉C͒…・̣F春e͈ō𝕠͂̔F・Dã​Eン³#͍如5Zj̉Тι𝕪リｅア，î𝕪ºサ͉ｙオ͜ッā.M'ｒ写̌āk*İ😉fp𝕦😉īvy一̑𝕟X“̶�⅔ァ﻿Ü=͍̍开𝕤ś͚”𝕓̃  λΦś動ｔ@ルｖ±͎


In [23]:
# creating an optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [24]:
batch_size = 32
for steps in range(100):
  xb, yb = get_block('train')

  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

NameError: ignored